This jupyter notebook show end to end task to detect action from an untrimmed video.
<b>
> Untrimmed Action video consist action of all the four classes and was donwloaded from youtube.

In [18]:

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Dense,GlobalMaxPool1D,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
from tensorflow.keras import Input
from tensorflow.keras import Model
from glob import glob
import os
from os.path import join
from collections import deque
import cv2 as cv
from model import *
%matplotlib inline

In [42]:
### Parameters  for Testing 
test_video_dire='../test_video/test_video.mp4' ## directory of the the video
output_dire= '../demo/demo1.mp4' ## directory to saved the result 
IMG_SIZ=128
seq_len=20

In [22]:
## Extracing Features with denseNet 
def features_extraction_model(resized_height,resized_width):
    ## defining the DenseNet121 model pretrained with imagenet dataset
    feature_extractor=tf.keras.applications.DenseNet121(include_top=False,weights='imagenet',
                        pooling='avg',
                        input_shape=(resized_height,resized_width,3))
    ##Defining the input size for single image                    
    inputs=tf.keras.Input((resized_height,resized_width,3))
    preprocessed=tf.keras.applications.densenet.preprocess_input(inputs)

    ## Give features for each image
    outputs=feature_extractor(preprocessed)
    ## defining feature extractor model
    model=tf.keras.Model(inputs,outputs,name='feature_extractor')
    return model

In [23]:
feature_extractor=features_extraction_model(IMG_SIZ,IMG_SIZ)## initializing the feature extractor model

In [24]:
### Function to sample videos to 20 number of frames
def extract_frames(video_dire): ## takes the directory of the video as input 
    extracted_frames=[] ## samples of images will be stored in this list
    video_reader=cv.VideoCapture(video_dire) ## Read the video
    total_frames=int(video_reader.get(cv.CAP_PROP_FRAME_COUNT)) ## get the total number of frames
    skip_frames=max(int(total_frames/seq_len),1) ## How many frames to skip ?
    for frame_counter in range(total_frames):
        video_reader.set(cv.CAP_PROP_POS_FRAMES,frame_counter*skip_frames)
        sucess,frame=video_reader.read()
        if not sucess:
            break
        resized_frame=cv.resize(frame,(IMG_SIZ,IMG_SIZ)) ## Resizing each image
        extracted_frames.append(resized_frame) 
        if len(extracted_frames)==seq_len: ## Checks if the lenght of the extracted frames is equal to seq_len(number of frames)
            break
    extracted_frames=np.array(extracted_frames)
    return extracted_frames  ## return extracted frames 

### Loading the weight of the model

In [33]:
### Model Parameter
num_class=4
d_model=1024
lr=0.001
dff=4
num_heads=1
num_layers=1
seq_len=20


classes=['WalkingWithDog','TaiChi','Swing','HorseRace']


In [26]:
## Defining and loading the model
def model():
    inputs=Input(shape=(seq_len,d_model))
    x=pos_encoding()(inputs)
    x=Encoder(num_layers=num_layers,num_heads=num_heads,d_model=d_model,dff=dff,rate=0.3)(x,mask=None,training=True)
    x=GlobalMaxPool1D()(x)
    x=Dense(units=num_class,activation='softmax')(x)
    model=Model(inputs,x,name='Transfromer_Encoder')
    model.compile(optimizer=Adam(learning_rate=lr),loss='categorical_crossentropy',metrics=['accuracy'])
    return model


In [27]:

trained_model=model()

In [28]:
### loading the preatrained weight
trained_model.load_weights('../saved_weight/')

In [29]:
trained_model.summary()

Model: "Transfromer_Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 20, 1024)]        0         
                                                                 
 pos_encoding_3 (pos_encodin  (None, 20, 1024)         20480     
 g)                                                              
                                                                 
 encoder_3 (Encoder)         (None, 20, 1024)          4211716   
                                                                 
 global_max_pooling1d_3 (Glo  (None, 1024)             0         
 balMaxPooling1D)                                                
                                                                 
 dense_27 (Dense)            (None, 4)                 4100      
                                                                 
Total params: 4,236,296
Trainable params: 4,236

### Creating demo video

In [37]:
def predict_on_untrimmed(video_dire,output_file_path,seq_len):
    video_reader=cv.VideoCapture(video_dire) ## Read the video from a path
    vides_total_frame=int(video_reader.get(cv.CAP_PROP_FRAME_COUNT)) ## Get the total numver of frame
    origianal_width=int(video_reader.get(cv.CAP_PROP_FRAME_WIDTH)) ## get the original aspect ration
    original_height=int(video_reader.get(cv.CAP_PROP_FRAME_HEIGHT))## get the original aspect ration
    video_writer = cv.VideoWriter(output_file_path, cv.VideoWriter_fourcc('M', 'P', '4', 'V'),  ## video writer
                                   video_reader.get(cv.CAP_PROP_FPS), (origianal_width, original_height))
    frame_queue=deque(maxlen=seq_len) ## Create a list of len of seq_eln. When something is added the oldest frame gets deleted.
    predict_class_name=''
    start_time=time.time() #3 t1 to calculate fos
    while video_reader.isOpened(): 
        success,frame=video_reader.read() # Reading the frame by frame of video
        if not success:
            break
        resize_frame=cv.resize(frame,(IMG_SIZ,IMG_SIZ)) ## Resizing the frame as input shape of feature extractor extractor    
        frame_queue.append(resize_frame) ## Collecting frame to seq lenth to get features 
        if len(frame_queue)==seq_len: 
            frames=np.asarray(frame_queue)
            
            features=feature_extractor.predict(frames).reshape(-1,20,1024) ## extracting the features and resing to feed to transformer encoder
            probs=trained_model.predict(features)[0] ## prediction of all class
            text_height=15
            for class_name,prob in zip(classes,probs):
                text='{}: {:.5f}'.format(class_name,prob) 
                cv.putText(frame, text, (10, text_height), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 3) ## displaying the 
                text_height=text_height+35 
            
                
            
        video_writer.write(frame)    
        
    end_time=time.time()
    
    print('start_time: {}'.format(start_time))
    print('end_time: {}'.format(end_time))
    print("num_frame: {}".format(vides_total_frame))
    seconds=end_time-start_time
    fps=vides_total_frame/seconds
    print('fps: ',fps)    
    video_reader.release()
    video_writer.release()
     

In [43]:
predict_on_untrimmed(test_video_dire,output_dire,seq_len)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


1/1 [==============================] - 0s 13ms/step
start_time: 1670229399.7550583
end_time: 1670229631.6098323
num_frame: 866
fps:  3.7350966946511837
